# Парсинг журнала ИДК

In [1]:
import re
import openpyxl as xl
import requests as req

In [2]:
def to_address(index):
    index = str(index)
    # вначале идут первые три цифры индекса, а потом весь
    post_url = r'https://www.postindexapi.ru/json/{}/{}.json'
    info = post_url.format(index[0:3], index)
    json = req.get(info).json()
    return (json['Region'], json['City'])

# to_address(357114)

## Чтение из Excel

In [3]:
import xlrd

In [4]:
# открыли файл
journal_name = 'Журнал контрактов 28.07.2020.xlsm'
workbook = xl.load_workbook(journal_name)

# открыли лист
sheet_name = 'действующие 2020'
sheet = workbook[sheet_name]

In [5]:
xl_field = {}

for (column_num, field_name) in enumerate(map(lambda x: x.value, sheet[1])):
    if type(field_name) == str and not field_name[:1:].isdigit():
        xl_field[field_name] = column_num

tmp = {}
for (column_num, field_name) in enumerate(map(lambda x: x.value, sheet[2])):
    if type(field_name) == str:
        tmp.setdefault(field_name, []).append(column_num)
        xl_field[field_name + ' ' + str(len(tmp[field_name]))] = column_num

In [6]:
xl_types = {
    '№п/п': 'Int64',
    'Наименование организации заказчика': 'string',
    'Адрес, контакт': 'string',
    '№ договора / контракта': 'string',
    'Срок': 'string',
    'Статус': 'string',
    'ИНН': 'Int64',
    'Внеш': 'Int64',
    'ДТУ': 'Int64',
    'ДТЛ': 'Int64',
    '№ прот 1': 'string',
    'Отб+Зак 1': 'string',
    'С/А 1': 'string',
    'сумма 1': 'float64',
    '№ прот 2': 'string',
    'Отб+Зак 2': 'string',
    'С/А 2': 'string',
    'сумма 2': 'float64',
    '№ прот 3': 'string',
    'Отб+Зак 3': 'string',
    'С/А 3': 'string',
    'сумма 3': 'float64',
    '№ прот 4': 'string',
    'Отб+Зак 4': 'string',
    'С/А 4': 'string',
    'сумма 4': 'float64'
}

In [7]:
import pandas as pd
import numpy as np

In [8]:
# уот так уот можно просто прочитать из экселя
xldf = pd.read_excel(journal_name, sheet_name=sheet_name,
                     dtype=xl_types,
                     names=xl_field.keys(), index_col=None,
                     usecols=xl_field.values(),
                     skiprows=1, nrows=153)

In [9]:
xldf.shape

(153, 26)

In [10]:
xldf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 26 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   №п/п                                151 non-null    Int64  
 1   Наименование организации заказчика  153 non-null    string 
 2   Адрес, контакт                      152 non-null    string 
 3   № договора / контракта              148 non-null    string 
 4   Срок                                133 non-null    string 
 5   Статус                              135 non-null    string 
 6   ИНН                                 150 non-null    Int64  
 7   Внеш                                0 non-null      Int64  
 8   ДТУ                                 85 non-null     Int64  
 9   ДТЛ                                 61 non-null     Int64  
 10  № прот 1                            111 non-null    string 
 11  Отб+Зак 1                           113 non-n

После того, как распарсили эксель, нужно подготовить данные к отправке в базу данных.

## База клиентов

In [11]:
def parse_contact_info(contact_info):
    address, postcode, contact = None, None, None
    try:
        address, contact = tuple(contact_info.split(';')[:2:])
    except Exception:
        address = contact_info
        contact = pd.NA
    
    try:
        postcode = int(address[:6:])
    except Exception:
        pass
    
    return address, postcode, contact

In [12]:
client = pd.DataFrame(columns=[
    'name', 'taxpayer_id'
])

client['name'] = xldf['Наименование организации заказчика']
client['taxpayer_id'] = xldf['ИНН']
    
client.head(15)

,name,taxpayer_id
0,ГБУЗ СК ГБ (ГП №2),2635221863
1,ГБУЗ СК ГБ (ГБ№2),<NA>
2,Дантист,<NA>
3,ГБУЗ СК ГБ (ГДБ),2631017874
4,"Невинномысский филиал АНМО ""СКККДЦ"", г.невинно...",2634049360
5,"ООО ""ЗУБР""",2631027142
6,"ООО ""Улыбка"", г.Невинномысск",2631028107
7,"ООО ""ЦСС"", г.Невинномысск",2631030603
8,"ООО ""Эстетическая Стоматология"" Невинномысск",2631027880
9,"ООО НППФ ""Альянс-Рем""",<NA>


## Информация по ИДК клиентам

In [13]:
idc_info = pd.DataFrame(columns=[
    'address', 'contact', 'postcode',
    'DTU_num', 'DTL_num'
])

idc_info['DTU_num'] = xldf['ДТУ']
idc_info['DTL_num'] = xldf['ДТЛ']

contact_info = xldf['Адрес, контакт'].apply(parse_contact_info)

idc_info['address'] =\
    contact_info.apply(lambda x: x[0]).astype('string')
idc_info['postcode'] =\
    contact_info.apply(lambda x: x[1]).astype('Int32')
idc_info['contact'] =\
    contact_info.apply(lambda x: x[2]).astype('string')

idc_info.head(15)

,address,contact,postcode,DTU_num,DTL_num
0,"357101, Ставропольский край, город Невинномысс...",8-928-818-28-30 Елена Валер...,357101,4,<NA>
1,"357112, Ставропольский край, город Невинномысс...",8-928-820-68-17 Гл. медсестра. Ирина Анатоль...,357112,3,<NA>
2,"357100, Ставропольский край, город Невинномысс...",Татьяна 8-918-876-48-05,357100,1,<NA>
3,"357113, Ставропольский край, город Невинномысс...",Ольга Гавриловна Рентген лаборант 8-919-754-...,357113,5,<NA>
4,"357107 Ставропольский край, г.Невинномысск, ул...",Галина Викторовна 8-905-496-20-13,357107,13,<NA>
5,"357100, Ставропольский край, город Невинномысс...",8-928-013-66-49,357100,1,<NA>
6,"357111, Ставропольский край, город Невинномысс...",Елена Владимировна 8-938-302-14-59 рентгенолог,357111,1,<NA>
7,"357111, Ставропольский край, город Невинномысс...",Спасова Ольга Александровна +7 (928)-316-4...,357111,3,<NA>
8,"357100, Ставропольский край, город Невинномысс...",8-928-264-63-14 Юля,357100,2,<NA>
9,<NA>,<NA>,<NA>,7,<NA>


## Квартальная отчетность

In [14]:
quarter = {
    1: xldf[['№ прот 1', 'Отб+Зак 1', 'С/А 1', 'сумма 1']],
    2: xldf[['№ прот 2', 'Отб+Зак 2', 'С/А 2', 'сумма 2']],
    3: xldf[['№ прот 3', 'Отб+Зак 3', 'С/А 3', 'сумма 3']],
    4: xldf[['№ прот 4', 'Отб+Зак 4', 'С/А 4', 'сумма 4']],
}

tmp = [
    'protocol_num',
    'docs_status',
    'bill_num',
    'tender'
]

quarter[1].columns = tmp
quarter[2].columns = tmp
quarter[3].columns = tmp
quarter[4].columns = tmp

quarter[1].info(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   protocol_num  111 non-null    string 
 1   docs_status   113 non-null    string 
 2   bill_num      108 non-null    string 
 3   tender        105 non-null    float64
dtypes: float64(1), string(3)
memory usage: 4.9 KB


## Договора

In [15]:
doc = pd.DataFrame(columns=[
    'client_id',
    'type',
    'num',
    'begin',
    'end'
])

Необходимо достать тип документа: договор или контракт, дату его начала и окончания.

In [16]:
doc_titles = xldf['№ договора / контракта']
doc_ends = xldf['Срок']
doc_id = 0
for (client_id, titles) in filter(lambda x: pd.notnull(x[1]),
                                     enumerate(doc_titles)):
    begin = pd.NA
    end = pd.NA
    types = list(map(lambda x: x.lower().title(),
                     re.findall(r'договор|контракт', titles, flags=re.I)))
    
    titles_lst = list(map(lambda x: re.sub(r'\s+|\n+', ' ', x.strip()),
                      filter(lambda x: len(x) > 0,
                             re.split(r'договор|контракт', titles, flags=re.I))))
    
    for (doc_type, doc_title) in zip(types, titles_lst):
        num = list(map(lambda x: re.sub(r'№|\bот\b', '', x).strip(),
                       re.findall(r'^.+от|^.+$', doc_title)))[0]
        
        dates = re.findall(r'\d{1,2}[\.|/|\-]\d\d[\.|/|\-]\d{4}', doc_title)
        
        if len(dates) > 0:
            begin = dates[0]
        end = doc_ends[client_id]
        
        if pd.notnull(end):
            end = re.sub(r'00:00:00', '', end)    
        elif len(dates) == 2:
            end = dates[1]
        doc.loc[doc_id] = [client_id, doc_type, num, begin, end]
        doc_id = doc_id + 1

Далее наводим порядок с форматами дат:

In [17]:
for doc_id, date in filter(lambda x: pd.notnull(x[1]) and x[1][:4:].isdigit(), enumerate(doc.end)):
    doc.end[doc_id] = '.'.join(date.strip().split('-')[::-1])
    
for doc_id, date in filter(lambda x: pd.notnull(x[1]), enumerate(doc.end)):
    doc.end[doc_id] = re.sub(r'г', '', date)

In [18]:
for doc_id, date in filter(lambda x: pd.notnull(x[1]), enumerate(doc.end)):
    doc.end[doc_id] = '-'.join(date.strip().split('.')[::-1])

In [19]:
for doc_id, date in filter(lambda x: pd.notnull(x[1]), enumerate(doc.begin)):
    doc.begin[doc_id] = '-'.join(date.strip().split('.')[::-1])

In [20]:
doc.begin = doc.begin.astype('datetime64')
doc.end = doc.end.astype('datetime64')
doc.num = doc.num.astype('string')
doc.type = doc.type.astype('string')
doc.client_id = doc.client_id.astype('Int32')

In [21]:
doc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167 entries, 0 to 166
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   client_id  167 non-null    Int32         
 1   type       167 non-null    string        
 2   num        167 non-null    string        
 3   begin      164 non-null    datetime64[ns]
 4   end        154 non-null    datetime64[ns]
dtypes: Int32(1), datetime64[ns](2), string(2)
memory usage: 12.3 KB


## Какие данные мне еще необходимы?

### Организации без ИНН

In [22]:
client[pd.isnull(client.taxpayer_id)]

,name,taxpayer_id
1,ГБУЗ СК ГБ (ГБ№2),<NA>
2,Дантист,<NA>
9,"ООО НППФ ""Альянс-Рем""",<NA>


### Организации без почтовых индексов

In [23]:
idc_info[pd.isnull(idc_info.postcode)].join(client)

,address,contact,postcode,DTU_num,DTL_num,name,taxpayer_id
9,<NA>,<NA>,<NA>,7,<NA>,"ООО НППФ ""Альянс-Рем""",<NA>
12,"Юр. адрес: 357111, Ставропольский край, город ...",Евгения 8-999-379-12-38,<NA>,<NA>,1,"ООО ""ЛадаДент""",2610017369
26,"Ставропольский край, г. Ставрополь, ул. Семашк...",Нередько Вадим Александрович Зав. отделением...,<NA>,15,<NA>,"ГБУЗ СК ""СККПЦ №1""",2635221863
35,"г. Ставрополь, ул. Ленина, д. 482/1, кв. 27",Летов Иван Иванович 8-903-445-20-85,<NA>,2,<NA>,ИП Летов И.И.,263401052767
69,"фактический адрес: 355042, Ставропольский край...",Михаил 8-962-447-04-88,<NA>,4,<NA>,"ООО ""Аксиома-Ставрополь""",2634102849
100,"Российская Федерация, Ставропольский край, г. ...",Оракова Зейнап Рашидовна тел. 8-86558-4-92-89,<NA>,2,<NA>,"ООО ""Газпром трансгаз Ставрополь"", Камыш-Бурун...",2636032629
107,"Юр. адрес: 355040, Россия,Ставропольский край,...",Маршалкин Филипп Анатольевич 8-928-962-447-3...,<NA>,<NA>,1,"ООО ""Виталиния М""",2635228241
118,"г.Симферополь, ул. 60 лет Октября, 30",,<NA>,<NA>,<NA>,ИП Гусарова Юлия Витальевна,910200227313
120,"г.Симферополь, Трубаченко 24",,<NA>,<NA>,<NA>,ИП Цеков Евгений Сергеевич,910200227634
121,"г. Симферополь, Трубаченко, 24",,<NA>,<NA>,<NA>,ИП Ткаченко Наталья Николаевна,910402859611


### Договора/контракты без даты начала/конца

In [24]:
doc[pd.isnull(doc.begin) | pd.isnull(doc.end)].join(client, on='client_id')

,client_id,type,num,begin,end,name,taxpayer_id
17,18,Договор,141/12-19 ИД,NaT,2020-12-31,"ГАУЗ СК ""ГСП № 2"" Г. Ставрополя",2636041790
29,30,Договор,430708,2020-03-02,NaT,"ГБУЗ СК ГКП №1, г.Ставрополя",2633003193
30,30,Договор,463287,2020-06-03,NaT,"ГБУЗ СК ГКП №1, г.Ставрополя",2633003193
31,30,Договор,492794,2020-09-09,NaT,"ГБУЗ СК ГКП №1, г.Ставрополя",2633003193
32,31,Договор,85/01-20 ИД 09.01.2020г (1квартал),2020-01-09,NaT,"ГБУЗ СК КСКИБ, г.Ставрополя",2633001990
49,45,Договор,75/12-18 ИД,2018-12-04,NaT,"ООО ""Лаврентьевой Х. Г.""",2636052167
64,61,Договор,Д 012001050,NaT,2021-01-20,"ООО ""Газпром трансгаз Ставрополь"", г.Ставрополь",2636032629
85,80,Договор,155/08-20 ИД,2020-08-19,NaT,"ГБУЗ СК ""Левокумская РБ"" , с.Левокумское",2613006870
97,92,Договор,384727,2019-12-10,NaT,"ГБУЗ СК ""Благодарненская РБ"" , г.Благодарный",2605010625
125,115,Договор,562906-20STV,NaT,NaT,"ГБУЗ СК ""Краевой Центр СВМП № 1"",г.Буденновск",2624028635


### Нет номера протокола

In [25]:
quarter[1][pd.isnull(quarter[1].protocol_num) & (
    pd.notnull(quarter[1].docs_status) | pd.notnull(quarter[1].tender) | pd.notnull(quarter[1].bill_num)
)].join(client)

,protocol_num,docs_status,bill_num,tender,name,taxpayer_id
57,<NA>,В/О,<NA>,NaN,"ФКУЗ ""МСЧ МВД России по Ставропольскому краю"" ...",2634069743
111,<NA>,З,<NA>,NaN,"ООО ""Династия""",2372009511
133,<NA>,З,<NA>,NaN,"ООО ""Раддент""",9102258090
134,<NA>,З,<NA>,NaN,ООО «ЭФАРМОН»,9204008156
150,<NA>,З,<NA>,NaN,"РГБУЗ ""КЦГРБ""",902009354


In [26]:
quarter[2][pd.isnull(quarter[2].protocol_num) & (
    pd.notnull(quarter[2].docs_status) | pd.notnull(quarter[2].tender) | pd.notnull(quarter[2].bill_num)
)].join(client)

,protocol_num,docs_status,bill_num,tender,name,taxpayer_id
138,<NA>,Д/З,<NA>,NaN,"ООО ""Медицинские Системы""",9102013703
150,<NA>,<NA>,<NA>,3000.0,"РГБУЗ ""КЦГРБ""",902009354


In [27]:
quarter[3][pd.isnull(quarter[3].protocol_num) & (
    pd.notnull(quarter[3].docs_status) | pd.notnull(quarter[3].tender) | pd.notnull(quarter[3].bill_num)
)].join(client)

,protocol_num,docs_status,bill_num,tender,name,taxpayer_id
35,<NA>,З,<NA>,NaN,ИП Летов И.И.,263401052767
110,<NA>,З,<NA>,NaN,"ООО ""Техмед""",2607018852
140,<NA>,З,<NA>,NaN,ООО «Семейная стоматология»,9111015295
144,<NA>,З,<NA>,NaN,"ООО ""Блеск""",6154138497
149,<NA>,<NA>,З/Д,NaN,"РГБУЗ ""ЧГКБ""",901027093


In [28]:
quarter[4][pd.isnull(quarter[4].protocol_num) & (
    pd.notnull(quarter[4].docs_status) | pd.notnull(quarter[4].tender) | pd.notnull(quarter[4].bill_num)
)].join(client)

,protocol_num,docs_status,bill_num,tender,name,taxpayer_id
102,<NA>,З,<NA>,NaN,ГБУЗ СК «Железноводская городская больница» ДО...,2627026682
122,<NA>,З,<NA>,NaN,Медилайн,9107041038
124,<NA>,З,<NA>,NaN,"ООО ""МРТ Керчь""",9111016299
131,<NA>,З,<NA>,NaN,"ООО ""3Д СТОМ""",9102255614
142,<NA>,З,<NA>,NaN,"ООО ""МРЦ ""Гармония""",9111004455
143,<NA>,З,<NA>,NaN,"ООО ""МРТ Феодосия""",9108111231
152,<NA>,Д/З,<NA>,NaN,"ООО ""Жемчужина""",917021065


# Структура и типы БД

In [29]:
client.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   name         153 non-null    string
 1   taxpayer_id  150 non-null    Int64 
dtypes: Int64(1), string(1)
memory usage: 2.7 KB


In [30]:
idc_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   address   152 non-null    string
 1   contact   147 non-null    string
 2   postcode  143 non-null    Int32 
 3   DTU_num   85 non-null     Int64 
 4   DTL_num   61 non-null     Int64 
dtypes: Int32(1), Int64(2), string(2)
memory usage: 6.0 KB


In [31]:
doc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167 entries, 0 to 166
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   client_id  167 non-null    Int32         
 1   type       167 non-null    string        
 2   num        167 non-null    string        
 3   begin      164 non-null    datetime64[ns]
 4   end        154 non-null    datetime64[ns]
dtypes: Int32(1), datetime64[ns](2), string(2)
memory usage: 12.3 KB


In [32]:
quarter[1].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   protocol_num  111 non-null    string 
 1   docs_status   113 non-null    string 
 2   bill_num      108 non-null    string 
 3   tender        105 non-null    float64
dtypes: float64(1), string(3)
memory usage: 4.9 KB
